# Deploy model

In [7]:
from azureml.core import Workspace
from azureml.core.model import Model

# Connect to workspace
ws = Workspace.from_config(path="./config/ws_config.json")


In [ ]:
# Register model
model = Model.register(workspace = ws,
                        model_path ="models/svc.pkl",
                        model_name = "svc",
                        tags = {"pkl": "demo"},
                        description = "SVC to predict the crowdness in an area",)

In [ ]:
# 
import json
import numpy as np
import os
from sklearn.externals import joblib
from sklearn.linear_model import Ridge

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType


def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment. Join this path with the filename of the model file.
    # It holds the path to the directory that contains the deployed model (./azureml-models/$MODEL_NAME/$VERSION).
    # If there are multiple models, this value is the path to the directory containing all deployed models (./azureml-models).
    # Alternatively: model_path = Model.get_model_path('sklearn_mnist')
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'svc.pkl')
    # Deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

def run(data):
    try:
        result = model.predict(data)
        # You can return any data type, as long as it is JSON serializable.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error